In [1]:
z%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.ticker
import IPython.display

# 이분법(bisection method)

In [2]:
def bisection_once(f, a, b, e=1e-6):
    fa, fb = f(a), f(b)
    error = b - a
    error = error/2
    c = a + error
    fc = f(c)
    if abs(error) < e:
        # convergence
        return a, b, True, fa, fb
    if np.sign(fa) != np.sign(fc):
        b = c
        fb = fc
    else:
        a = c
        fa = fc
    return a, b, False, fa, fb

def bisection(f, a, b, nmax=20, e=1e-6):
    fa, fb = f(a), f(b)
    if np.sign(fa) == np.sign(fb):
        return
    error = b - a
    for n in range(nmax + 1):
        error = error/2
        c = a + error
        fc = f(c)
        if abs(error) < e:
            # convergence
            return a, b, True, fa, fb
        
        if np.sign(fa) != np.sign(fc):
            b = c
            fb = fc
        else:
            a = c
            fa = fc
    return a, b, False, fa, fb
        

In [3]:
fig, ax = plt.subplots()
ax.set_xlim((0-0.05, 1+0.05))

domain = np.linspace(0, 1, 1000)
codomain = np.linspace(-1, 1, 10)

ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))
ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
a = 0
b = 1
f = lambda x: x**3-3*x+1
T = [(a, b, (a+b)/2)]
for i in range(50):
    a, b, isconv, fa, fb = bisection_once(f, a, b)
    T.append((a, b, (a+b)/2))
    if isconv:
        print("convergence value:", (a+b)/2)
        break

ax.plot(domain, domain**3-3*domain+1, 'g')
ax.plot(domain, np.zeros_like(domain), 'black')
_la, = plt.plot([], [], 'b')
_lb, = plt.plot([], [], 'b')
_lc, = plt.plot([], [], 'r')
def animate(frame):
    _a, _b, _c = T[frame]
    _la.set_data(np.full_like(codomain, _a), codomain)
    _lb.set_data(np.full_like(codomain, _b), codomain)
    _lc.set_data(np.full_like(codomain, _c), codomain)
    ax.set_title(f"mean: {_c}")
    return _la, _lb, _lc

anim = animation.FuncAnimation(fig, animate, frames=len(T), interval=500)
plt.close()
IPython.display.HTML(anim.to_html5_video())

convergence value: 0.34729671478271484


In [3]:
fig, ax = plt.subplots()
ax.set_xlim((0.5-0.05, 2+0.05))
ax.set_ylim((-3, 3))
domain = np.linspace(0.5, 2, 1000)
codomain = np.linspace(-3, 3, 10)

ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.1))
ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
a = 0.5
b = 2
f = lambda x: x**3-2*np.sin(x)
T = [(a, b, (a+b)/2)]
for i in range(50):
    a, b, isconv, fa, fb = bisection_once(f, a, b)
    T.append((a, b, (a+b)/2))
    if isconv:
        print("convergence value:", (a+b)/2)
        break

ax.plot(domain, domain**3-2*np.sin(domain), 'g')
ax.plot(domain, np.zeros_like(domain), 'black')
_la, = plt.plot([], [], 'b')
_lb, = plt.plot([], [], 'b')
_lc, = plt.plot([], [], 'r')
def animate(frame):
    _a, _b, _c = T[frame]
    _la.set_data(np.full_like(codomain, _a), codomain)
    _lb.set_data(np.full_like(codomain, _b), codomain)
    _lc.set_data(np.full_like(codomain, _c), codomain)
    ax.set_title(f"mean: {_c}")
    return _la, _lb, _lc

anim = animation.FuncAnimation(fig, animate, frames=len(T), interval=500)
plt.close()
IPython.display.HTML(anim.to_html5_video())

convergence value: 1.2361834049224854


# 가위치법

In [5]:
def falsepos_once(f, a, b, e=1e-6):
    fa, fb = f(a), f(b)
    c = (a*fb - b * fa) / (fb - fa)
    fc = f(c)
    if abs(fc) < e:
        return a, b, True, fa, fb
    if np.sign(fa) != np.sign(fc):
        b = c
        fb = fc
    else:
        a = c
        fa = fc
    return a, b, False, fa, fb

def falsepos(f, a, b, nmax=20, e=1e-6):
    fa, fb = f(a), f(b)
    if np.sign(fa) == np.sign(fb):
        return

    for n in range(nmax + 1):
        c = (a*fb - b*fa) / (fb - fa)
        fc = f(c)
        if abs(fc) < e:
            return a, b, True, fa, fb
        if np.sign(fa) != np.sign(fc):
            b = c
            fb = fc
        else:
            a = c
            fa = fc

    return a, b, False, fa, fb


In [6]:
fig, ax = plt.subplots()

domain = np.linspace(-3.5, -1.5, 1000)
codomain = np.linspace(-3, 4, 10)

a = -3.5
b = -1.5
f = lambda x: (x+4)*(x+2)*(x-6)
T = [(a, b)]

def cc(a, b, fa, fb):
    if abs(fa) < abs(fb):
        return a, fa
    else:
        return b, fb

for i in range(50):
    a, b, isconv, fa, fb = falsepos_once(f, a, b)
    T.append((a, b))
    if isconv:
        print("convergence value:", cc(a, b, fa, fb)[0])
        break

ax.plot(domain, f(domain))
ax.plot(domain, np.zeros_like(domain), 'black')

_la, = plt.plot([], [], 'b')
_lb, = plt.plot([], [], 'b')
def animate(frame):
    _a, _b, = T[frame]
    _la.set_data(np.full_like(codomain, _a), codomain)
    _lb.set_data(np.full_like(codomain, _b), codomain)

    ax.set_title(f"convergence: {cc(_a, _b, f(_a), f(_b))[0]}")
    return _la, _lb

anim = animation.FuncAnimation(fig, animate, frames=len(T), interval=500)
plt.close()
IPython.display.HTML(anim.to_html5_video())

convergence value: -2.0000002070557996


# 개선된 가위치법

반복적으로 동일한 끝점이 선택되는 상황
이는 선형 수렴성을 저하시킬 수 있다.
따라서 같은 끝점이 두 번 선택되면 다음 식을 사용
$$c_k^{(m)} = \begin{cases}\frac{a_k f(b_k) - 2b_k f(a_k)}{f(b_k)-2f(a_k)} & (f(a_k)f(b_k) < 0) \\ \frac{2a_k f(b_k) - b_kf(a_k)}{2f(b_k)-f(a_k)} & (f(a_k)f(b_k) > 0) \end{cases}$$

# 뉴턴법 (또는 뉴턴-랩슨 반복법)

## 뉴턴법의 이해
두 함수 $f$가 미분 가능하다고 가정. 즉, $f$가 각 점에서 **유일한 접선**을 가진다.

$x_0$근방에서 다음과 같은 선형함수가 주어진 함수 $f$에 가깝다.
$$\ell(x) = f'(x_0)(x-x_0)+f(x_0)$$

$f$의 근에 대한 근사로 $\ell$의 근을 택한다.

$$x_1 = x_0 - \frac{f(x_0)}{f'(x_0)}$$
따라서 점 $x_0$에서 ㅡ시작하여 위 공식에서 새로운 점 $x_1$을 얻을 수 있다.

이 과정을 반복함으로써 순서대로 점을 구할 수 있다.

적절한 조건에서 이러한 점들의 수열은 $f$의 근으로 수렴한다.

## 뉴턴법의 또다른 이해
$$f(x_0 + h) = 0$$

$f$가 충분히 다루기 쉬운 함수라면 $x_0$에서 테일러 급수가 존재할 것.

$$f(x_0)+hf'(x_0)+\frac{h^2}{2}f''(x_0)+\dots = 0$$

위식에서 첫 두 항만 제외하고 모두 무시하면 $h$의 근사를 얻을 수 있다.

$$x_{n+1}=x_n - \frac{f(x_n)}{f'(x_n)}$$

이제 이게 성립할까?

$$\lim_{n\to \infty}{x_n} = r$$

$r$은 찾고자 하는 $f$의 근


# 할선법

$$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}$$

$f'(x_n)$을 쉽게 계산할 수 있는 근사값으로 바꾼다.
도함수의 정의가 다음과 같으므로
$$f'(x) = \lim_{h\to 0}{\frac{f(x+h)-f(x)}{h}}$$

아주 작은 $h$에 대해 다음이 성립한다.

$$f'(x) \approx \frac{f(x+h)-f(x)}{h}$$

특히 $x = x_n$이고 $h=x_{n-1} - x_n$인 경우, 다음이 성립한다.

$$f'(x_n) \approx \frac{f(x_{n-1}-f(x_n)}{x_{n-1} - x_n}$$

이 근사를 처음 식에 대입하면 그것이 할선법이다.

$$x_{n+1} = x_n - \left(\frac{x_n-x_{n-1}}{f(x_n)-f(x_{n-1}}f(x_n) \right)$$



